In [1]:
import duckdb

In [2]:
con = duckdb.connect(":memory:")

# Filtered sample

In [7]:
con.execute("SELECT COUNT(*) FROM '../data/wildchat.parquet' WHERE country = 'United States'").fetchone()

(275084,)

In [8]:
con.execute("SELECT COUNT(DISTINCT conversation_hash) FROM '../data/wildchat.parquet' WHERE country = 'United States'").fetchdf()

,count(DISTINCT conversation_hash)
0,145041


# Random annotation sample

In [12]:
con.execute("SELECT classification, COUNT(*) FROM '../data/sample_for_annotation_annotated.csv' GROUP BY classification").fetchdf()

,classification,count_star()
0,0,992
1,1,8


In [11]:
con.execute("SELECT * FROM '../data/sample_for_annotation_annotated.csv' WHERE classification IS NULL").fetchdf()

,conversation_hash,content,classification
0,d578616152e1f877fc84909163b87d49,﻿\n\nSolve the system with elimination.\n8x + ...,NaN


# Targeted search sample

In [16]:
with open("../data/searched_news.txt", "r") as f:
    records = f.read()

print(len(records.split("---")) - 1)

58
